In [19]:
# 데이터 생성
import pandas as pd
import numpy as np

# Seed for reproducibility
np.random.seed(42)

# Generate x values
x = np.linspace(-10, 10, 100)

# Generate random noise
noise = np.random.uniform(0.01, 0.99, x.shape)

# Calculate y values
y = 3.1 * x**2 - 1.7 * x + noise

# Create a DataFrame
data = pd.DataFrame({'x': x, 'y': y})

# Save to a CSV file
file_path = '../datasets/non_linear.csv'
data.to_csv(file_path, index=False)

file_path

'../datasets/non_linear.csv'

In [20]:
# 라이브러리 및 프레임워크 초기화 (3.35)

import torch
import pandas as pd # non_linear.csv를 읽기 위해 사용
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader

In [21]:
# 사용자 정의 데이터세트 (3.36)

class CustomDataset(Dataset):
    # 초기화 메서드
    def __init__(self, file_path):
        df = pd.read_csv(file_path) #파일 경로 지정
        self.x = df.iloc[:, 0].values #파일에서 데이터 x값과 y값을 할당하며, 데이터의 전체 길이도 저장함
        self.y = df.iloc[:, 1].values
        self.length = len(df)

    # 호출 메서드 : x와 y값을 반환하며, 이차 방정식 "y = w1 x**2 + w2 x + b" 형태임
    def __getitem__(self, index):
        x = torch.FloatTensor([self.x[index] ** 2, self.x[index]])
        y = torch.FloatTensor([self.y[index]])
        return x, y

    # 길이 반환 메서드 : 현재 데이터의 길이 제공
    def __len__(self):
        return self.length

In [22]:
# 사용자 정의 모델 (3.37)
class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Linear(2, 1)

    def forward(self, x):
        x = self.layer(x)
        return x

In [23]:
train_dataset = CustomDataset("../datasets/non_linear.csv")
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True, drop_last=True)

In [24]:

device = "cuda" if torch.cuda.is_available() else "cpu"
model = CustomModel().to(device)
criterion = nn.MSELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.0001)

In [25]:
for epoch in range(10000):
    cost = 0.0

    if len(train_dataloader) == 0:
        print("train_dataloader is empty. Exiting the training loop.")
        break

    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)

        output = model(x)
        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    cost = cost / len(train_dataloader)

    if (epoch + 1) % 1000 == 0:
        print(f"Epoch : {epoch+1:4d}, Model : {list(model.parameters())}, Cost : {cost:.3f}")


train_dataloader is empty. Exiting the training loop.


In [26]:
with torch.no_grad():
    model.eval()
    inputs = torch.FloatTensor(
        [
            [1 ** 2, 1],
            [5 ** 2, 5],
            [11 ** 2, 11]
        ]
    ).to(device)
    outputs = model(inputs)
    print(outputs)

tensor([[ -0.1369],
        [ -2.3976],
        [-18.3683]], device='cuda:0')


In [27]:
torch.save(
    model,
    "../models/model.pt"
)

torch.save(
    model.state_dict(),
    "../models/model_state_dict.pt"
)